In [1]:
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications import inception_v3
from IPython import display
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import librosa
from pathlib import Path
import os, shutil
plt.ioff()

In [2]:
!rm -rf data
!mkdir data
!mkdir data/dataset

"rm" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
Sottodirectory o file data gi… esistente.
Sintassi del comando errata.


In [3]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '1jwIWW2vuBJVO-XcCTL9HgmcolCfQJ2ir'    
download_file_from_google_drive(file_id, "data/data.zip")

In [4]:
from zipfile import ZipFile
zf = ZipFile('data/data.zip', 'r')
zf.extractall('data/dataset')
zf.close()

In [5]:
data_dir = pathlib.Path("data/dataset/data_final")

In [6]:
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis=-1)

In [7]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)

  # Note: You'll use indexing here instead of tuple unpacking to enable this 
  # to work in a TensorFlow graph.
  return parts[-2].numpy().decode()

In [8]:
def get_waveform(file_path):
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform, audio_binary

In [9]:
frame_length = 255
frame_step = 255
num_mel_bins = 75
num_spectrogram_bins = (frame_length // 2) + 1
fmin = 0.0
sample_rate = 44100
fmax = sample_rate / 2


def get_spectrogram(waveform):
    # Padding for files with less than 16000 samples
    zero_padding = tf.zeros([140000] - tf.shape(waveform), dtype=tf.float32)
    # Concatenate audio with padding so that all audio clips will be of the 
    # same length
    waveform = tf.cast(waveform, tf.float32)
    equal_length = tf.concat([waveform, zero_padding], 0)
    magnitude_spectrograms  = tf.signal.stft(
      equal_length, frame_length, frame_step)
    magnitude_spectrograms  = tf.abs(magnitude_spectrograms)
    
    # Step: magnitude_spectrograms->mel_spectrograms
    # Warp the linear-scale, magnitude spectrograms into the mel-scale.
    num_spectrogram_bins = magnitude_spectrograms.shape[-1]


    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins, num_spectrogram_bins, sample_rate, fmin,
        fmax)

    mel_spectrograms = tf.tensordot(
        magnitude_spectrograms, linear_to_mel_weight_matrix, 1)

    mel_spectrograms.set_shape(magnitude_spectrograms.shape[:-1].concatenate(
  linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    # Compute MFCCs from log_mel_spectrograms and take the first 13.
    #mfccs = tf.signal.mfccs_from_log_mel_spectrograms(
    #  log_mel_spectrograms)[..., :75]
  
    return log_mel_spectrograms

In [10]:
def plot_spectrogram(spectrogram, label, class_path, i):
  fig, ax = plt.subplots(figsize=(20,20))
  mfcc_data= np.swapaxes(spectrogram, 0 ,1)
  cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower')
  ax.axis("off")
  fig.savefig('data\dataset\mels{}\mel_{}_{}.png'.format(class_path.split("data\dataset\data_final")[1], label, i), bbox_inches='tight', pad_inches=0, dpi=300)


Main

In [11]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np
import soundfile as sf
import librosa


augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    #TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    #Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
])


In [12]:
def augmentation(path):
    y, sr = librosa.load(path)
    augmented_samples = augment(samples=y, sample_rate=sr)
    return augmented_samples, sr

In [ ]:
main_dir = "data/dataset/data_final"
folders_pathlist = Path(main_dir).glob('*')

for path in folders_pathlist:
    # because path is object not string
    class_path = str(path)
    print(class_path)
    wav_pathlist = Path(class_path).glob('./' + ('[0-9]' * 2) + ".wav")
    i = 300
    for w_path in wav_pathlist:
        print(w_path)
        wav_path = str(w_path)
        label = get_label(wav_path)
        augmented_samples, sr = augmentation(wav_path)
        print('data/dataset/data_final/{}/{}.wav'.format(label,i))
        sf.write('data/dataset/data_final/{}/{}.wav'.format(label,i), augmented_samples, sr)
        i +=1


data\dataset\data_final\Alces_alces
data\dataset\data_final\Alces_alces\01.wav
data/dataset/data_final/Alces_alces/300.wav
data\dataset\data_final\Alces_alces\02.wav
data/dataset/data_final/Alces_alces/301.wav
data\dataset\data_final\Alces_alces\03.wav
data/dataset/data_final/Alces_alces/302.wav
data\dataset\data_final\Alces_alces\04.wav
data/dataset/data_final/Alces_alces/303.wav
data\dataset\data_final\Alces_alces\05.wav
data/dataset/data_final/Alces_alces/304.wav
data\dataset\data_final\Alces_alces\06.wav
data/dataset/data_final/Alces_alces/305.wav
data\dataset\data_final\Alces_alces\07.wav
data/dataset/data_final/Alces_alces/306.wav
data\dataset\data_final\Alces_alces\08.wav
data/dataset/data_final/Alces_alces/307.wav
data\dataset\data_final\Alces_alces\09.wav
data/dataset/data_final/Alces_alces/308.wav
data\dataset\data_final\Alces_alces\10.wav
data/dataset/data_final/Alces_alces/309.wav
data\dataset\data_final\Alces_alces\11.wav
data/dataset/data_final/Alces_alces/310.wav
data\da

In [ ]:

try:
    os.makedirs('data/dataset/mels')
except:
    print("Folder already exists, deleting its content to replace them with new ones.")
    shutil.rmtree('data/dataset/mels')
    os.makedirs('data/dataset/mels')

In [ ]:
main_dir = "data/dataset/data_final"
folders_pathlist = Path(main_dir).glob('*')

for path in folders_pathlist:
    # because path is object not string
    class_path = str(path)
    print(class_path)
    wav_pathlist = Path(class_path).glob('*')
    i = 0
    os.makedirs('data/dataset/mels/{}'.format(class_path.rsplit("data_final\\")[1]))
    for w_path in wav_pathlist:
        wav_path = str(w_path)
        wave, _ = get_waveform(wav_path)
        label = get_label(wav_path)
        mel = get_spectrogram(wave)
        plot_spectrogram(mel, label, class_path, i)
        i +=1


In [ ]:
#Divide dataset in train val test

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio("data/dataset/mels", output="data/dataset/mel_final", seed=1337, ratio=(.8, .1, .1)) # default values

In [ ]:
tf.shape(wave)